**Beam Search**

In [9]:
import heapq

# Define the graph as an adjacency list with edge costs
graph = {
    'S': [('A', 3), ('B', 6), ('C', 5)],
    'A': [('D', 9), ('E', 8)],
    'B': [('F', 12), ('G', 14)],
    'C': [('H', 7)],
    'H': [('I', 5), ('J', 6)],
    'I': [('K', 1), ('L', 10), ('M', 2)],
    'D': [], 'E': [], 'F': [], 'G': [], 'J': [],
    'K': [], 'L': [], 'M': []  # Leaf nodes
}

# Beam Search function
def beam_search(start, goal, beam_width=2):
    # Initialize the beam with the start state
    beam = [(0, [start])]  # (cumulative cost, path)

    while beam:
        candidates = []

        # Expand each path in the beam
        for cost, path in beam:
            current_node = path[-1]
            if current_node == goal:
                return path, cost  # Return the path and cost if goal is reached

            # Generate successors
            for neighbor, edge_cost in graph.get(current_node, []):
                new_cost = cost + edge_cost
                new_path = path + [neighbor]
                # print(current_node, new_path)
                candidates.append((new_cost, new_path))

        # Select top-k paths based on the lowest cumulative cost
        beam = heapq.nsmallest(beam_width, candidates, key=lambda x: x[0])
        # print(beam)
        # print("can: ", candidates)
    return None, float('inf')  # Return None if no path is found

# Run Beam Search
start_node = 'S'
goal_node = 'L'
beam_width = 3
path, cost = beam_search(start=start_node, goal=goal_node, beam_width=beam_width)

# Print results
if path:
    print(f"Path found: {' → '.join(path)} with total cost: {cost}")
else:
    print("No path found.")


Path found: S → C → H → I → L with total cost: 27


**Simple Hill Climbing**

In [1]:
import random

# Heuristic function: Counts the number of pairs of attacking queens
def calculate_conflicts(state):

    conflicts = 0
    n = len(state)
    for i in range(n):
        for j in range(i + 1, n):
            # Check same column or diagonal
            if state[i] == state[j] or abs(state[i] - state[j]) == abs(i - j):
                conflicts += 1
    return conflicts

# Generate neighbors by moving one queen at a time
def get_neighbors(state):
    neighbors = []
    n = len(state)
    for row in range(n):
        for col in range(n):
            if col != state[row]:
                new_state = list(state)
                new_state[row] = col
                neighbors.append(new_state)
    return neighbors

# Simple Hill Climbing function
def simple_hill_climbing(n):
    # Random initial state
    current_state = [random.randint(0, n - 1) for _ in range(n)]
    current_conflicts = calculate_conflicts(current_state)

    while True:
        neighbors = get_neighbors(current_state)
        next_state = None
        next_conflicts = current_conflicts

        # Find the first better neighbor
        for neighbor in neighbors:
            neighbor_conflicts = calculate_conflicts(neighbor)
            if neighbor_conflicts < next_conflicts:
                next_state = neighbor
                next_conflicts = neighbor_conflicts
                break  # Move to the first better neighbor

        # If no better neighbor is found, return the current state
        if next_conflicts >= current_conflicts:
            break

        # Move to the better neighbor
        current_state = next_state
        current_conflicts = next_conflicts

    return current_state, current_conflicts

# Run Simple Hill Climbing for N-Queens
n = 8  # Change N here for different sizes
solution, conflicts = simple_hill_climbing(8)

# Print results
if conflicts == 0:
    print(f"Solution found for {n}-Queens problem:")
    print(solution)
else:
    print(f"Could not find a solution. Stuck at state with {conflicts} conflicts:")
    print(solution)

Solution found for 8-Queens problem:
[3, 7, 0, 2, 5, 1, 6, 4]


**GA**

In [2]:
import random

# Define the number of queens
n = 8

# Fitness function: counts non-attacking pairs of queens
def calculate_fitness(individual):
    non_attacking_pairs = 0
    total_pairs = n * (n - 1) // 2  # Maximum possible non-attacking pairs

    # Check for conflicts
    for i in range(n):
        for j in range(i + 1, n):
            # No same column or diagonal conflict
            if individual[i] != individual[j] and abs(individual[i] - individual[j]) != abs(i - j):
                non_attacking_pairs += 1

    # Fitness score is the ratio of non-attacking pairs
    return non_attacking_pairs / total_pairs


In [3]:
# Generate a random individual (chromosome) based on column positions
def create_random_individual():
    return random.sample(range(n), n)  # Ensure unique column positions

# Create an initial population of random individuals
population_size = 10
population = [create_random_individual() for _ in range(population_size)]

In [4]:
# Evaluate fitness for each individual
fitness_scores = [calculate_fitness(ind) for ind in population]
print("Fitness Scores:", fitness_scores)

Fitness Scores: [0.8571428571428571, 0.7857142857142857, 0.8214285714285714, 0.9642857142857143, 0.7857142857142857, 0.75, 0.8571428571428571, 0.7857142857142857, 0.8928571428571429, 0.8214285714285714]


In [5]:
# Select parents based on fitness
def select_parents(population, fitness_scores):
    sorted_population = [route for _, route in sorted(zip(fitness_scores, population), reverse=True)]
    return sorted_population[:len(population) // 2]

# Select parents
parents = select_parents(population, fitness_scores)
print("Selected Parents:", parents)


Selected Parents: [[5, 0, 2, 4, 7, 1, 3, 6], [0, 4, 3, 7, 5, 2, 6, 1], [7, 5, 2, 4, 0, 3, 1, 6], [6, 0, 5, 3, 2, 7, 4, 1], [7, 3, 2, 5, 4, 1, 0, 6]]


In [6]:
# Crossover function: single-point crossover with unique column positions
def crossover(parent1, parent2):
    point = random.randint(1, n - 2)  # Choose a crossover point
    child = parent1[:point] + parent2[point:]

    # Ensure unique column positions
    missing = set(range(n)) - set(child)
    duplicates = [col for col in child if child.count(col) > 1]
    for i in range(len(child)):
        if child.count(child[i]) > 1:
            child[i] = missing.pop()
    return child

# Create new population using crossover
new_population = []
for _ in range(population_size):
    parent1, parent2 = random.sample(parents, 2)
    child = crossover(parent1, parent2)
    new_population.append(child)
print("New Population after Crossover:", new_population)


New Population after Crossover: [[5, 0, 6, 3, 2, 7, 4, 1], [6, 0, 5, 3, 2, 7, 4, 1], [0, 2, 3, 6, 5, 7, 4, 1], [7, 3, 2, 5, 4, 0, 1, 6], [0, 5, 3, 6, 2, 7, 4, 1], [0, 5, 6, 3, 2, 7, 4, 1], [4, 0, 5, 7, 2, 1, 3, 6], [7, 3, 2, 5, 4, 1, 0, 6], [7, 3, 0, 5, 4, 2, 6, 1], [7, 5, 2, 4, 3, 1, 0, 6]]


In [7]:
# Mutation function: swap two column positions
def mutate(individual):
    idx1, idx2 = random.sample(range(n), 2)
    individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual

# Apply mutation with a probability of 0.1
mutation_rate = 0.1
for i in range(len(new_population)):
    if random.random() < mutation_rate:
        new_population[i] = mutate(new_population[i])
print("Population after Mutation:", new_population)


Population after Mutation: [[5, 0, 6, 3, 2, 7, 4, 1], [6, 0, 5, 3, 2, 7, 4, 1], [0, 2, 3, 6, 5, 7, 1, 4], [7, 3, 2, 5, 4, 0, 1, 6], [0, 5, 3, 6, 2, 7, 4, 1], [0, 5, 6, 3, 2, 7, 4, 1], [4, 0, 5, 7, 2, 1, 3, 6], [7, 3, 2, 5, 4, 1, 0, 6], [0, 3, 7, 5, 4, 2, 6, 1], [7, 5, 2, 4, 3, 1, 0, 6]]


In [8]:
# Genetic Algorithm main function
def genetic_algorithm():
    population = [create_random_individual() for _ in range(population_size)]
    generation = 0
    best_fitness = 0

    while best_fitness < 1.0 and generation < 100:
        fitness_scores = [calculate_fitness(ind) for ind in population]
        best_fitness = max(fitness_scores)
        print(f"Generation {generation} Best Fitness: {best_fitness}")

        # Check for optimal solution
        if best_fitness == 1.0:
            break

        # Selection
        parents = select_parents(population, fitness_scores)

        # Crossover
        new_population = [crossover(random.choice(parents), random.choice(parents)) for _ in range(population_size)]

        # Mutation
        for i in range(len(new_population)):
            if random.random() < mutation_rate:
                new_population[i] = mutate(new_population[i])

        population = new_population
        generation += 1

    # Return the best solution
    best_individual = max(population, key=calculate_fitness)
    return best_individual, calculate_fitness(best_individual)

# Run the Genetic Algorithm
solution, fitness = genetic_algorithm()
print("Best Solution:", solution)
print("Best Fitness:", fitness)


Generation 0 Best Fitness: 0.9642857142857143
Generation 1 Best Fitness: 0.9642857142857143
Generation 2 Best Fitness: 0.9642857142857143
Generation 3 Best Fitness: 0.9642857142857143
Generation 4 Best Fitness: 0.9642857142857143
Generation 5 Best Fitness: 0.9642857142857143
Generation 6 Best Fitness: 0.9642857142857143
Generation 7 Best Fitness: 0.9642857142857143
Generation 8 Best Fitness: 0.9642857142857143
Generation 9 Best Fitness: 0.9642857142857143
Generation 10 Best Fitness: 0.9642857142857143
Generation 11 Best Fitness: 0.9642857142857143
Generation 12 Best Fitness: 0.9642857142857143
Generation 13 Best Fitness: 0.9642857142857143
Generation 14 Best Fitness: 0.9642857142857143
Generation 15 Best Fitness: 0.9642857142857143
Generation 16 Best Fitness: 0.9642857142857143
Generation 17 Best Fitness: 0.9642857142857143
Generation 18 Best Fitness: 0.9642857142857143
Generation 19 Best Fitness: 0.9642857142857143
Generation 20 Best Fitness: 0.9642857142857143
Generation 21 Best Fitn